In [1]:
import pandas as pd
import os
import datetime
import numpy as np

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/QC_new_sign_ups_every_week'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
Q1_new_sign_ups=list(recursive_file_gen("/home/jian/BigLots/"))
Q1_new_sign_ups=[x for x in Q1_new_sign_ups if "Master" in x]
Q1_new_sign_ups=[x for x in Q1_new_sign_ups if "2019" in x]

In [4]:
df_Q1_new_sign_ups=pd.DataFrame({"master_file_path":Q1_new_sign_ups})
df_Q1_new_sign_ups['week_end_dt']=df_Q1_new_sign_ups['master_file_path'].apply(lambda x: x.split("_weeks/MediaStorm_")[1][:10])
df_Q1_new_sign_ups=df_Q1_new_sign_ups[(df_Q1_new_sign_ups['week_end_dt']>="2019-02-09") & (df_Q1_new_sign_ups['week_end_dt']<="2019-05-04")]
len(df_Q1_new_sign_ups)

13

In [5]:
Q1_daily=list(recursive_file_gen("/home/jian/BigLots/"))
Q1_daily=[x for x in Q1_daily if "aily" in x]
Q1_daily=[x for x in Q1_daily if "2019" in x]
Q1_daily=[x for x in Q1_daily if "isto" not in x]
Q1_daily=[x for x in Q1_daily if "txt" in x]

In [6]:
df_Q1_daily=pd.DataFrame({"daily_file_path":Q1_daily})
df_Q1_daily['week_end_dt']=df_Q1_daily['daily_file_path'].apply(lambda x: x.split("eeks/MediaStorm_")[1][:10])
df_Q1_daily=df_Q1_daily[(df_Q1_daily['week_end_dt']>="2019-02-09") & (df_Q1_daily['week_end_dt']<="2019-05-04")]
len(df_Q1_daily)

13

In [7]:
df_file_path=pd.merge(df_Q1_daily,df_Q1_new_sign_ups,on="week_end_dt")

In [14]:
df_summary=pd.DataFrame()
i_counter=0
for index,row in df_file_path.iterrows():
    master_path=row['master_file_path']
    daily_path=row['daily_file_path']
    week_end_dt=row['week_end_dt']
    
    df_master=pd.read_csv(master_path,sep="|",dtype=str,usecols=['customer_id_hashed','email_address_hash','sign_up_date','sign_up_channel','sign_up_location','transaction_count','transaction_amount'])
    df_master['transaction_count']=df_master['transaction_count'].astype(float).astype(int)
    df_master['transaction_amount']=df_master['transaction_amount'].astype(float).astype(int)
    
    df_daily=pd.read_csv(daily_path,sep="|",dtype=str,usecols=['customer_id_hashed','transaction_dt']).drop_duplicates()
    df_daily=df_daily[~pd.isnull(df_daily['customer_id_hashed'])]
    df_both=pd.merge(df_master,df_daily,on="customer_id_hashed",how="left")
    
    id_total=df_master['customer_id_hashed'].nunique()
    id_shopped=df_both[~pd.isnull(df_both['transaction_dt'])]['customer_id_hashed'].nunique()
    
    df_summary=df_summary.append(pd.DataFrame({"week_end_dt":week_end_dt,"id_total":id_total,"id_shopped":id_shopped},index=[i_counter]))
    i_counter+=1

In [34]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/QC_new_sign_ups_every_week/BL_new_sign_up_master_files_id_count_QA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary=df_summary[['week_end_dt','id_total','id_shopped']]
df_summary.to_excel(writer,"New_Rewards_Count_QC",index=False)
writer.save()


In [23]:
df_daily_0504_new=df_daily_0504[pd.notnull(df_daily_0504['email_address_hash'])]

df_daily_0504_new['item_transaction_amt']=df_daily_0504_new['item_transaction_amt'].astype(float)

df_daily_0504_new_agg=df_daily_0504_new.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
df_daily_0504_new_agg.head(4)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,customer_id_hashed,item_transaction_amt
0,0001118b56efc2a9245479d4bab50e5cb6e196fa590194...,12.00
1,000184a43747fdcd881a2eee3f124e67845b7962458271...,1339.97
2,00026ab41f5f272e0a8c678b364c86483af82bc02d3b99...,259.99
3,0002be91e93417ee6707ad3b905db5441ab8733181e608...,17.25


In [24]:
df_daily_0504_new_agg.shape

(82965, 2)

In [26]:
df_daily_0504_new_agg['item_transaction_amt'].max()

4047.7599999999989

In [32]:
df_daily_0504_new_agg[df_daily_0504_new_agg['item_transaction_amt']<=20].shape

(29008, 2)

In [33]:
29008/84615

0.342823376469893